In [1]:
print 'load libraries...'

load libraries...


In [2]:
# The purpose of this project is to analysis retail credits and create credit scores for all individuals
# A number of retail loans open to customers were used for selection
# Customers with these existing products with latest loan date after jan2016 were extracted
# Based on a NPL list from ERM customers (active) were labelled as good and bad


In [3]:
# Add path in order to load utility functions
import sys
sys.path.append(r"N:\18. Hot Spot\Custom Modules\Utils")
sys.path.append(r"N:\18. Hot Spot\Custom Modules\ETLMonthly")


In [4]:
# setup environment and import libraries
import os
import re
import datetime
import time
import getpass
import pandas as pd
import numpy as np
from datetime import datetime, date
from datetime import timedelta
import calendar as cd
from collections import OrderedDict
import sqlalchemy
import sqlalchemy
from sqlalchemy import types, create_engine
import file_utils
import db_utils
import config

path=r'C:\ProgramData\Anaconda2\instantclient_12_2'
os.environ['PATH']=path
import cx_Oracle

#set working directory
os.chdir(config.working_dir)

In [5]:
# db credentials authentication

In [6]:
print 'connect to database...'
conn_string = db_utils.db_connect(config.db_username, config.db_pwd)

connect to database...
Testing Connection...
Connection Succesful!


In [7]:
mtch = re.match('.*@', conn_string)
un, pwd = mtch.group().rstrip('@').split('/')

In [8]:
TARGET_VAR='IS_BAD'

In [9]:
LOAN_START_DATE = '01jan2019'

In [10]:
tbl_nm='cs_retail_loan_cust'

In [11]:
# # Extract the retail loan ledgers that will be considered during the analysis
# retail_loan_ledgers = pd.read_excel(r"..\Docs\Retail loan ledgers.xlsx", sheetname="SQL Results")['LED_CODE']
# retail_loan_ledgers = ','.join([str(l) for l in retail_loan_ledgers])

In [12]:
qc_retail_loan_ledgers = [1118,1121]

#### Get NPL Customer List From Retail Promo

In [13]:
print 'get bad customers from NPL list...'

# read excel file
df_npl_list = pd.concat( \
        [pd.read_excel(r'..\Docs\All dates NPLs.xlsx', sheetname='Sheet1', usecols=['LOANACCT','NO_OF_DAYS_IN_OD']), \
         pd.read_excel(r'..\Docs\All dates NPLs.xlsx', sheetname='Sheet2', usecols=['LOANACCT','NO_OF_DAYS_IN_OD'])] \
    ) 

get bad customers from NPL list...


In [14]:
# split account key into components
df_npl_list[['BRA_CODE','CUS_NUM','CUR_CODE','LED_CODE','SUB_ACCT_CODE']] = df_npl_list.LOANACCT.str.split('/', expand=True)

In [15]:
# remove LOANACCT column
del df_npl_list['LOANACCT']

In [16]:
# convert account key components to number
df_npl_list[['BRA_CODE','CUS_NUM','CUR_CODE','LED_CODE','SUB_ACCT_CODE']] = \
    df_npl_list[['BRA_CODE','CUS_NUM','CUR_CODE','LED_CODE','SUB_ACCT_CODE']].astype(int)

In [17]:
df_npl_list = df_npl_list.groupby(['BRA_CODE','CUS_NUM','CUR_CODE','LED_CODE','SUB_ACCT_CODE'], 
                                  as_index=False).agg({'NO_OF_DAYS_IN_OD':max})

In [18]:
df_npl_list[TARGET_VAR]=1

In [19]:
print 'Number of rows in NPL list: ', len(df_npl_list)

Number of rows in retail NPL list:  47789


In [20]:
print 'unique NPL cust all loan types: ', len(df_npl_list[['BRA_CODE','CUS_NUM','IS_BAD']].drop_duplicates())

unique NPL cust all loan types:  40965


In [21]:
# len(df_npl_list_cust)

In [22]:
# 20,859 - old count

In [23]:
# df_npl_list[df_npl_list.duplicated(subset=['BRA_CODE','CUS_NUM','CUR_CODE','LED_CODE','SUB_ACCT_CODE'], keep=False)].sort_values(
# by=['BRA_CODE','CUS_NUM','CUR_CODE','LED_CODE','SUB_ACCT_CODE'])

#### Extract all retail loan customer

In [24]:
# df_retail_active_loans = pd.read_csv('df_retail_active_loans_original.csv')

In [27]:
print 'get all retail loan customers from database...'

con = cx_Oracle.connect(conn_string)
df_retail_active_loans = pd.read_sql_query('''
select bra_code, cus_num, cur_code, led_code, sub_acct_code, date_open,
trunc(to_date('{ref_date}'))-date_open as days_from_disbursement
from
(select a.bra_code, a.cus_num, a.cur_code, a.led_Code, a.sub_acct_code, a.date_open,
case when
  a.date_open=max(a.date_open) over(partition by a.bra_code, a.cus_num) and
  a.sub_acct_code=max(a.sub_acct_code) over(partition by a.bra_code, a.cus_num)
then 1 else 0 end is_latest_loan
from (select * from stg.d_account_snapshot where reference_date = '{ref_date}') a
inner join stg.src_cust_pro b
on a.bra_code=b.bra_code and a.cus_num=b.cus_num
where
b.cust_grade_seg in (1,2,3,4,5)
and a.cus_num>99999
and a.sta_code=1 and a.cus_num>99999 and a.date_open >= '{LOAN_START_DATE}'
and a.led_code in ({ledgers_string}))
where is_latest_loan=1
'''.format(ledgers_string=str(qc_retail_loan_ledgers)[1:-1], LOAN_START_DATE=LOAN_START_DATE, ref_date=config.ref_date), con)
con.close()

df_retail_active_loans.to_csv('df_retail_active_loans_original.csv', index=False)
print len(df_retail_active_loans)

get all retail loan customers from database...
219765


In [28]:
df_retail_active_loans_combined = df_retail_active_loans.merge(df_npl_list, how='left', 
                                                      on=['BRA_CODE','CUS_NUM','CUR_CODE','LED_CODE','SUB_ACCT_CODE'])

In [29]:
df_retail_active_loans_combined.fillna(0, inplace=True)

In [30]:
df_retail_active_loans_combined.IS_BAD.value_counts()

0.0    188660
1.0     31105
Name: IS_BAD, dtype: int64

In [31]:
df_retail_active_loans_combined[df_retail_active_loans_combined.duplicated(
    subset=['BRA_CODE','CUS_NUM','CUR_CODE','LED_CODE','SUB_ACCT_CODE'], keep=False)]

,BRA_CODE,CUS_NUM,CUR_CODE,LED_CODE,SUB_ACCT_CODE,DATE_OPEN,DAYS_FROM_DISBURSEMENT,NO_OF_DAYS_IN_OD,IS_BAD


In [32]:
# Check if duplicates exist by: account keys
assert sum(df_retail_active_loans_combined.duplicated(
    subset=['BRA_CODE','CUS_NUM','CUR_CODE','LED_CODE','SUB_ACCT_CODE']))==0, 'Warning: Duplicated loan accounts present'

In [33]:
# Check if duplicates exist by: customer keys
assert sum(df_retail_active_loans_combined.duplicated(
    subset=['BRA_CODE','CUS_NUM']))==0, 'Warning: Duplicated loan customers present'

In [34]:
# check some customers
df_retail_active_loans_combined[(df_retail_active_loans_combined.BRA_CODE==205) & (df_retail_active_loans_combined.CUS_NUM==158905)]

,BRA_CODE,CUS_NUM,CUR_CODE,LED_CODE,SUB_ACCT_CODE,DATE_OPEN,DAYS_FROM_DISBURSEMENT,NO_OF_DAYS_IN_OD,IS_BAD
9191,205,158905,1,1118,1,2019-11-23,72,0.0,0.0


#### Attach status (IS_BAD) to retail loans

In [35]:
is_bad_select_cond = (df_retail_active_loans_combined.IS_BAD==1) & (df_retail_active_loans_combined['NO_OF_DAYS_IN_OD']>=90)
is_good_select_cond = (df_retail_active_loans_combined.IS_BAD==0) & (df_retail_active_loans_combined['DAYS_FROM_DISBURSEMENT']>=90)

In [36]:
df_retail_loan_cust = df_retail_active_loans_combined[is_bad_select_cond | is_good_select_cond]

In [37]:
df_retail_loan_cust.IS_BAD.value_counts()

0.0    141783
1.0     12941
Name: IS_BAD, dtype: int64

In [38]:
######### old figures ########
# 0.0    170195
# 1.0      6281
# Name: IS_BAD, dtype: int64

#### Load data

In [39]:
df_retail_loan_cust.IS_BAD.value_counts()

0.0    141783
1.0     12941
Name: IS_BAD, dtype: int64

In [40]:
# df_retail_loan_cust = pd.read_csv('cs_retail_loan_cust.csv')

In [41]:
df_retail_loan_cust['DATE_OPEN'] = pd.to_datetime(df_retail_loan_cust.DATE_OPEN)

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [42]:
df_retail_loan_cust.to_csv('cs_retail_loan_cust.csv', index=False)

In [43]:
conn = create_engine('oracle+cx_oracle://{}:{}@dwdrscan.gtbank.com:1521/?service_name=odidb'.format(un, pwd))
df_retail_loan_cust.to_sql('cs_retail_loan_cust', conn, index=False, if_exists='replace', chunksize=10000, 
                           dtype=db_utils.sqlcol(df_retail_loan_cust))

In [44]:
df_retail_loan_cust.IS_BAD.value_counts()

0.0    141783
1.0     12941
Name: IS_BAD, dtype: int64

In [45]:
print min(df_retail_loan_cust.DATE_OPEN)
print max(df_retail_loan_cust.DATE_OPEN)

2019-02-12 00:00:00
2019-11-05 00:00:00
